[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/databyjp/AcademyXI_DA/blob/main/notebooks/AcademyXi_DA_Module_7_data_analysis_1.ipynb)

## AcademyXi Data Analysis - Data Analysis 1: 
### Workshop - Data analysis with Python
In this workshop module, we will show you how to use Python to perform some the data analysis tasks you've already seen.

In [1]:
# Install additional libraries required (fsspec and s3fs) to load files through AWS S3
%%capture tmp
!pip install fsspec s3fs

# Import libraries to be used
import pandas as pd
import numpy as np
import plotly.express as px

#### Visualising distributions

Below reproduces many of the charts from earlier in the module with Plotly. Although you have already seen how to produce statistical visualisations such as histograms, these may be useful as revision material.

In [2]:
df = pd.read_csv("s3://databyjp/academyxi/wk7_physPerformance.csv")

In [ ]:
df.head()

In [ ]:
# Histogram - see https://plotly.com/python/histograms/
fig = px.histogram(df, x="height_cm", nbins=40)
fig.show()

In [ ]:
fig = px.histogram(df, x="gripForce", nbins=40)
fig.show()

In [ ]:
fig = px.histogram(df, x="gripForce", nbins=40, color="gender")
fig.show()

In [ ]:
# Overlaid histogram - https://plotly.com/python/histograms/#overlaid-histogram
fig = px.histogram(df, x="gripForce", nbins=40, color="gender")
fig.update_layout(barmode='overlay')  
fig.update_traces(opacity=0.75)
fig.show()

In [ ]:
fig = px.scatter(df, x="weight_kg", y="height_cm")
fig.show()

In [ ]:
fig = px.scatter(df, x="weight_kg", y="broad jump_cm")
fig.show()

In [ ]:
fig = px.scatter(df, x="weight_kg", y="broad jump_cm", color="gender")
fig.show()

#### Scatter plot matrices (aka Pairplots)

Now that you know how to use scatter plots to identify correlations, you may wish to start visualising a number of combinations of variables. 

This task is made easier with tools which can produce scatter plot matrics, also known as pairplots.

- https://plotly.com/python/splom/
- https://seaborn.pydata.org/generated/seaborn.pairplot.html

See an example below

In [ ]:
fig = px.scatter_matrix(df,
                        dimensions=["age", "height_cm", "diastolic", "gripForce", "broad jump_cm"], 
                        color="gender")
fig.show()

Although crude due to resolution limitations, these plots give some idea of correlation between variables at a glance, which may help to narrow down your search.

In almost all visualisation tools, you can adjust marker sizes. Where there are too many overlapping markers, plotting a randomly sampled subset may also help to reveal global patterns.

In [ ]:
sm_df = df.sample(frac=0.05)  # Create a dataframe with only a small portion of the full size dataframe
fig = px.scatter_matrix(sm_df,
                        dimensions=["age", "height_cm", "diastolic", "gripForce", "broad jump_cm"], 
                        color="gender")
fig.update_traces(marker=dict(size=2))  # Set a small size for the markers to better see patterns
fig.show()

## Grouping data

In pandas, grouping data is a two-step process comprising of using the `groupby` method to create a grouped data object, and then using one or more aggregation functions such as:
- `count`
- `sum`
- `min`
- `max`

To generate summaries of the grouped data. The resulting outputs are similar to what is possible through pivot tables.

Let's now open the Melbourne housing dataset which you would have seen before:

In [13]:
df = pd.read_csv("s3://databyjp/academyxi/wk7_melbourne_housing.csv")

In [ ]:
df.head()

We see in our casual inspection that the `Price` column includes null values. Let's clean the data by dropping those rows.

In [15]:
df = df[df["Price"].notna()]

Now, we can find which suburb's listings have the most or fewest rooms on average. This can be done by:
- Grouping the data by `Suburb` column, and
- Taking the mean of the values

In [ ]:
grp_df = df.groupby("Suburb")  # Group the dataframe
mean_rms = grp_df.mean()["Rooms"]  # Take a mean
print(mean_rms.sort_values().round(2))  # Sort by values, round to 2 decimal places, then print to screen

Now see if you can do the same, but get the mean prices, grouped by number of rooms

In [ ]:
# How would you modify the below to group the data by rooms, and print mean prices?
# Note: .astype(int) converts the resulting data to integers
print(df.groupby("Suburb").mean()["Rooms"].sort_values().astype(int))

To group the data by multiple columns, the input should be a python list as shown below. This will produce a grouped pandas series with a nested index (multi-index).

In [18]:
tmp_ser = df.groupby(["Bedroom2", "Bathroom"]).mean()["Price"].astype(int)

In [ ]:
tmp_ser.head()

The series can be converted a dataframe by the `.reset_index` method:

In [ ]:
grp_df = tmp_ser.reset_index()
grp_df.head()

It can be filtered just as we had previously done.

In [21]:
grp_df = grp_df[(grp_df["Bathroom"] > 0) & (grp_df["Bedroom2"] <= 3)]

In [ ]:
grp_df

#### Data bins

Bins of data can be created by pandas' `cut` or `qcut` methods to convert continuous variables to discrete ones. `cut` will create bins of given sizes, whereas `qcut` will create  bins based on number of observations in each bin.

Documentation:
- https://pandas.pydata.org/docs/reference/api/pandas.cut.html
- https://pandas.pydata.org/docs/reference/api/pandas.qcut.html

In [23]:
cut_bins = [i for i in range(0, 51, 5)]  # Create bins (0 to 50 in steps of 5) 
dist_bins = pd.cut(df["Distance"], bins=cut_bins)
df = df.assign(dist_bins=dist_bins)

In [ ]:
df.groupby("dist_bins").mean()["Price"].astype(int)

## Multiple aggregations

For aggregate multiple columns, pandas provides the `agg` method which allows aggregating different columns as well as different methods within. 

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.agg.html

In [ ]:
df.groupby("dist_bins").agg({"Price": "mean", "Suburb": "count"}).astype(int)

## Combining tables

Pandas provides functionalities to allow multiple tables to be joined. The main method with which to do this is its `merge` method. 
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html

Take a look at the below examples:

In [ ]:
pop_df = pd.read_csv("s3://databyjp/academyxi/wk7_wdi_pop_simple.csv")
co2_df = pd.read_csv("s3://databyjp/academyxi/wk7_wdi_co2_simple.csv")

In [ ]:
pop_df.head()

In [ ]:
co2_df.head()

To create an inner join on the `Country Code` column, the following syntax can be used.

In [ ]:
merged_df = pop_df.merge(co2_df, how="inner", on="Country Code")

In [ ]:
merged_df.head()

Now, let's see what would happen with dataframes containing slightly different indices. This can be created in our case by dropping null data rows, as each dataset is missing data for different countries.

Once the missing data has been dropped, we can see that the number of observations are different.

In [ ]:
co2_df = co2_df[co2_df["CO2 emissions (kt)"].notna()]
pop_df = pop_df[pop_df["Population, total"].notna()]
print(f"co2_df is {len(co2_df)} rows long")
print(f"pop_df is {len(pop_df)} rows long")

Now, take a look at the differences produced by:
- Inner join,
- Left join,
- Right join, and
- Outer join

In [ ]:
inner_merged_df = pop_df.merge(co2_df, how="inner", on="Country Code")

In [ ]:
inner_merged_df.head()

In [ ]:
left_merged_df = pop_df.merge(co2_df, how="left", on="Country Code")

In [ ]:
left_merged_df.head()

In [ ]:
right_merged_df = pop_df.merge(co2_df, how="right", on="Country Code")

In [ ]:
right_merged_df.head()

In [ ]:
outer_merged_df = pop_df.merge(co2_df, how="outer", on="Country Code")

In [ ]:
outer_merged_df.head()

Now we can check the number of rows in each dataframe:

In [ ]:
print(f"Inner merged dataframe: {len(inner_merged_df)} rows long")
print(f"Left merged dataframe: {len(left_merged_df)} rows long")
print(f"Right merged dataframe: {len(right_merged_df)} rows long")
print(f"Outer merged dataframe: {len(outer_merged_df)} rows long")

You can see that inner join only includes rows which are present in both dataframes, while left/right join only includes rows of the respective dataframes. 

The outer join includes rows that are present in at least one of the two dataframes.

-----

That's it for this tutorial. There's a lot more which you can do with Python in relation to exploratory data analysis as well as merging dataframes, but we hope this gave you an introduction. Feel free to adapt this notebook to different datasets, or search other Python modules and examples for EDA.